In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
import torch.nn.functional as F
import transformer_flow

from dataclasses import dataclass

import utils


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



/scratch/gpfs/EKOLEMEN/nc1514/ml-tarflow/.venv/lib/python3.10/site-packages/IPython/core/completerlib.py:150: UserWarning: This is now an optional IPython functionality, setting rootmodules_cache requires you to install the `pickleshare` library.
  ip.db['rootmodules_cache'] = rootmodules_cache


In [2]:
@dataclass
class Args:
    channel_size: int = 3
    img_size: int = 128
    patch_size: int = 8
    channels: int = 768
    blocks: int = 8
    layers_per_block: int = 8
    nvp: bool = True
    num_classes: int = 1000
    resume: str = 'afhq_model_8_768_8_8_0.07.pth'
    lr: float = 1e-4

# Compatible class
# args = Args(
#     img_size=256,
# )

args = Args()

In [3]:
model = transformer_flow.Model(
    in_channels=args.channel_size,
    img_size=args.img_size,
    patch_size=args.patch_size,
    channels=args.channels,
    num_blocks=args.blocks,
    layers_per_block=args.layers_per_block,
    nvp=args.nvp,
    num_classes=args.num_classes,
).to('cuda')

In [4]:
ckpt = torch.load(args.resume, map_location='cuda')

/tmp/ipykernel_3635912/1230320305.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(args.resume, map_location='cuda')


In [5]:
optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.95), lr=args.lr, weight_decay=1e-4)
lr_schedule = utils.CosineLRSchedule(optimizer, 1000, 3000, 1e-6, args.lr)

In [6]:
for i in range(args.blocks):
    print(f"Model: {model.blocks[i].pos_embed.shape}")
    print(f"Checkpoint: {ckpt[f'blocks.{i}.pos_embed'].shape}")

Model: torch.Size([256, 768])
Checkpoint: torch.Size([1024, 768])
Model: torch.Size([256, 768])
Checkpoint: torch.Size([1024, 768])
Model: torch.Size([256, 768])
Checkpoint: torch.Size([1024, 768])
Model: torch.Size([256, 768])
Checkpoint: torch.Size([1024, 768])
Model: torch.Size([256, 768])
Checkpoint: torch.Size([1024, 768])
Model: torch.Size([256, 768])
Checkpoint: torch.Size([1024, 768])
Model: torch.Size([256, 768])
Checkpoint: torch.Size([1024, 768])
Model: torch.Size([256, 768])
Checkpoint: torch.Size([1024, 768])


In [14]:
def _load_state_dict_custom(model, ckpt):
    
    # 1. Handle Sequence Length (Interpolation)
    old_num_patches = ckpt['blocks.0.pos_embed'].shape[0]
    new_num_patches = model.num_patches
    old_grid = int(old_num_patches**0.5)
    new_grid = int(new_num_patches**0.5)
    
    filtered_dict = {}
    for k, v in ckpt.items():
        # Interpolate Position Embeddings
        if 'pos_embed' in k:
            pos_tokens = v.unsqueeze(0).transpose(1, 2).reshape(1, -1, old_grid, old_grid)
            interpolated = torch.nn.functional.interpolate(
                pos_tokens, size=(new_grid, new_grid), mode='bicubic', align_corners=False
            )
            v = interpolated.reshape(1, -1, new_num_patches).transpose(1, 2).squeeze(0)
            
        # Interpolate Prior Variance (var)
        elif k == 'var':
            var_tokens = v.unsqueeze(0).transpose(1, 2).reshape(1, -1, old_grid, old_grid)
            interpolated_var = torch.nn.functional.interpolate(
                var_tokens, size=(new_grid, new_grid), mode='bicubic'
            )
            v = interpolated_var.reshape(1, -1, new_num_patches).transpose(1, 2).squeeze(0)

        # Skip incompatible class embeddings and buffers
        if 'class_embed' in k:
            print(f"Skipping {k}: AFHQ(3) -> ImageNet(1000)")
            continue
        if 'attn_mask' in k:
            continue # Use the mask already in your 128x128 model
            
        filtered_dict[k] = v

    # 2. Load with strict=False to allow class_embed to stay initialized for ImageNet
    msg = model.load_state_dict(filtered_dict, strict=False)
    print(f"Load Result: {msg}")
    return model

In [26]:
ckpt = torch.load(args.resume, map_location='cpu', weights_only=True)

In [27]:
model_2 = _load_state_dict_custom(model, ckpt)

Skipping blocks.0.class_embed: AFHQ(3) -> ImageNet(1000)
Skipping blocks.1.class_embed: AFHQ(3) -> ImageNet(1000)
Skipping blocks.2.class_embed: AFHQ(3) -> ImageNet(1000)
Skipping blocks.3.class_embed: AFHQ(3) -> ImageNet(1000)
Skipping blocks.4.class_embed: AFHQ(3) -> ImageNet(1000)
Skipping blocks.5.class_embed: AFHQ(3) -> ImageNet(1000)
Skipping blocks.6.class_embed: AFHQ(3) -> ImageNet(1000)
Skipping blocks.7.class_embed: AFHQ(3) -> ImageNet(1000)
Load Result: _IncompatibleKeys(missing_keys=['blocks.0.class_embed', 'blocks.0.attn_mask', 'blocks.1.class_embed', 'blocks.1.attn_mask', 'blocks.2.class_embed', 'blocks.2.attn_mask', 'blocks.3.class_embed', 'blocks.3.attn_mask', 'blocks.4.class_embed', 'blocks.4.attn_mask', 'blocks.5.class_embed', 'blocks.5.attn_mask', 'blocks.6.class_embed', 'blocks.6.attn_mask', 'blocks.7.class_embed', 'blocks.7.attn_mask'], unexpected_keys=[])


In [23]:
# ckpt = torch.load(args.resume.replace('_model_', '_opt_'), map_location='cpu')
optimizer.load_state_dict(ckpt['optimizer'])
lr_schedule.load_state_dict(ckpt['lr_schedule'])

KeyError: 'optimizer'